# Hopfield model with Hebb rule - single pattern

Based partly on https://scholarworks.sjsu.edu/cgi/viewcontent.cgi?article=1292&context=etd_theses, but mainly on the book by Herz, Krogh, etc. 

We have the Hopfield model, which consists of the dynamical evolution of patterns $xi_i$ given by
$$
\xi^{new}_i = {\rm sign} \left(\sum_{j=1}^N w_{ij} \xi_j^{old}\right)
$$
where $N$ is the number of units in the network; along with the Hebb rule which allows for learning $w_{ij}$ in the model as
$$
w_{ij} = \frac{1}{N}\sum_{s=1}^M \xi_i^{s}\xi_j^{s}
$$
where $M$ is the number of patterns to store. In our case we want a single pattern to be stored, therefore $M = 1$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import copy

## 0) Input-output

In [2]:
pattern = "-------------\n-----###-----\n----##-##----\n---##---##---\n--##-----##--\n--#########--\n--##-----##--\n--##-----##--\n-------------"

print(pattern)

-------------
-----###-----
----##-##----
---##---##---
--##-----##--
--#########--
--##-----##--
--##-----##--
-------------


In [3]:
L = len(pattern.split('\n')[0])
print(L)

13


In [4]:
def encode_pattern(pattern):
    '''Turn the text pattern into an array of zeros and ones.'''
    xi = []
    for lett in pattern:
        if lett == '-':
            xi.append(-1)
        elif lett == '#':
            xi.append(+1)

    return np.array(xi)

xi = encode_pattern(pattern)
print(xi)

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1  1  1 -1 -1 -1
 -1 -1 -1 -1 -1 -1  1  1 -1  1  1 -1 -1 -1 -1 -1 -1 -1  1  1 -1 -1 -1  1
  1 -1 -1 -1 -1 -1  1  1 -1 -1 -1 -1 -1  1  1 -1 -1 -1 -1  1  1  1  1  1
  1  1  1  1 -1 -1 -1 -1  1  1 -1 -1 -1 -1 -1  1  1 -1 -1 -1 -1  1  1 -1
 -1 -1 -1 -1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


In [5]:
def decode_pattern(array, L=13):
    '''Turn a zeros and ones array into a text pattern.'''
    
    text = ''
    n = 0
    
    for num in array:
        if num == -1:
            text += '-'
        elif num == +1:
            text += '#'
        else:
            text += '0'
            
        n += 1
        if n % L == 0:
            text += '\n'
            
    text = text[:-1]
            
    return text

text = decode_pattern(xi)

print(text)

-------------
-----###-----
----##-##----
---##---##---
--##-----##--
--#########--
--##-----##--
--##-----##--
-------------


## 1) The model

In [6]:
W = np.outer(xi,xi)/len(xi)# - np.identity(len(xi))

In [7]:
def Hopfield(xi_old, W):
    '''
    Returns a new pattern computed from the sign function of T times the old pattern
    '''
    
    xi_new = np.sign(W @ xi_old)
    
    return xi_new

Self-consistency relation:

In [8]:
xi_new = Hopfield(xi, W)

text = decode_pattern(xi_new)
print(text)

-------------
-----###-----
----##-##----
---##---##---
--##-----##--
--#########--
--##-----##--
--##-----##--
-------------


### 1.1) Scrambling the input array

In [9]:
def scramble(xi, p):
    '''
    Given an array xi and a percentage 0 <= p <= 1,
    return another array where 100*p% of it has been randomly changed.
    0 -> no change
    1 -> all digits changed
    '''
    
    xi_s = []
    
    for num in xi:
        if random.random() < p:
            if num == -1:
                xi_s.append(+1)
            if num == +1:
                xi_s.append(-1)
        else:
            xi_s.append(num)
                
    return np.array(xi_s)

In [10]:
xi_s1 = scramble(xi, 0.1)
xi - xi_s1

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -2,  0,  0,  0,  0,
        0,  0,  0,  0, -2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -2,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  2,  0,  0,  0,  0,  2,
        0,  0,  0,  0,  0,  0,  2,  0,  0, -2,  0,  0,  0,  2,  0,  0,  0,
        0,  0,  2,  0, -2,  0,  0,  0,  0,  0,  0,  0, -2,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0, -2,  0,  0,  0,  0,  0,  0])

In [11]:
xi_s5 = scramble(xi, 0.5)
xi - xi_s5

array([ 0, -2,  0,  0,  0,  0, -2, -2,  0,  0, -2, -2, -2, -2, -2, -2,  0,
        0,  2,  2,  0, -2,  0,  0, -2,  0, -2, -2,  0, -2,  0,  2, -2,  2,
        2,  0, -2,  0, -2,  0, -2,  0,  0,  0,  0, -2,  0,  2,  2,  0,  0,
       -2,  0, -2,  2,  2, -2, -2,  0,  0,  0,  2,  2, -2, -2, -2, -2,  0,
        0,  0,  2,  2,  2,  2,  0,  0,  0, -2, -2, -2,  0,  0, -2,  0,  0,
        0,  0,  0,  0, -2,  0, -2,  0,  2,  0,  0, -2, -2,  0,  0,  0,  2,
        0,  0, -2, -2, -2,  0, -2,  0,  0, -2,  0, -2,  0,  0, -2])

In [12]:
xi_s9 = scramble(xi, 0.9)
xi - xi_s9

array([-2, -2, -2, -2,  0, -2, -2, -2, -2,  0, -2, -2, -2, -2, -2, -2, -2,
       -2,  2,  2,  2, -2, -2,  0, -2, -2, -2, -2, -2, -2,  2,  2, -2,  2,
        2, -2, -2, -2, -2, -2, -2, -2,  2,  2, -2, -2,  0,  2,  2, -2,  0,
       -2, -2, -2,  0,  2,  0, -2, -2, -2, -2,  2,  2, -2, -2, -2, -2,  2,
        2,  2,  2,  2,  2,  0,  2,  2, -2,  0, -2, -2,  2,  2, -2,  0, -2,
       -2, -2,  0,  2, -2, -2, -2, -2,  2,  2, -2, -2, -2, -2, -2,  0,  2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2])

Time to see the attractor in action

In [13]:
xi_new = Hopfield(xi_s1, W)

text = decode_pattern(xi_new)
print(text)

-------------
-----###-----
----##-##----
---##---##---
--##-----##--
--#########--
--##-----##--
--##-----##--
-------------


In [14]:
xi_new = Hopfield(xi_s5, W)

text = decode_pattern(xi_new)
print(text)

#############
#####---#####
####--#--####
###--###--###
##--#####--##
##---------##
##--#####--##
##--#####--##
#############


In [15]:
xi_new = Hopfield(xi_s9, W)

pattern_rev = decode_pattern(xi_new)
print(pattern_rev)

#############
#####---#####
####--#--####
###--###--###
##--#####--##
##---------##
##--#####--##
##--#####--##
#############


This last example is interesting: it shows that this model actually has two attractors, the learned patter and its "inverse". This is already described in the literature.

### 1.2) Fully random input array

In [16]:
def random_input(xi):
    '''Generate a random array of +1, -1'''
    
    xi_r = []
    
    for i in range(len(xi)):
        num = random.choice((-1,+1))
        xi_r.append(num)
        
    return np.array(xi_r)

In [17]:
for i in range(10):
    xi_r = random_input(xi)

    xi_new = Hopfield(xi_r, W)

    text = decode_pattern(xi_new)
    
    if text == pattern:
        print('Text = pattern')
    elif text == pattern_rev:
        print('Text = reverse pattern')
    else:
        print(list(text))
        print(list(pattern))
        print(list(pattern_rev))
        raise Exception('Weeeird')

Text = pattern
Text = reverse pattern
Text = pattern
Text = pattern
Text = pattern
Text = reverse pattern
Text = reverse pattern
Text = pattern
Text = pattern
Text = reverse pattern


## 2) Dilution

Now we consider a fraction $c$ of the connections removed at random, and see the performance. We will study asymmetric dilutions.

In [18]:
def Hopfield_diluted(xi_old, W, c):
    '''
    Returns a new pattern computed from the sign function of T times the old pattern
    '''
    
    Wc = copy.copy(W)
    
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            
            if random.random() < c:
                Wc[i,j] = 0
    
    xi_new = np.sign(Wc @ xi_old)
    
    return xi_new

To have a clear understanding of how the capacity is being affected, we will put many outputs side by side with the following function.

In [19]:
def concatenate_scrambled_diluted(xi_s, N, W, c, H=9):
    '''Concatenate the output of several diluted networks with random inputs'''
    
    pat = [''] * H
    
    for i in range(N):
        xi_new = Hopfield_diluted(xi_s, W, c)
        text = decode_pattern(xi_new)
        text_split = text.split('\n')
        
        for line in range(H):
            pat[line] += '|'
            pat[line] += text_split[line]
        
        totaltext = '\n'.join(pat)
        
    return totaltext

#### Weak dilution, scrambled arrays:

In [20]:
totaltext = concatenate_scrambled_diluted(xi_s1, 10, W, 0.1)
print(totaltext)

|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------
|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----
|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----
|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---
|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--
|--#########--|--#########--|--#########--|--#########--|--#########--|--#########--|--#########--|--#########--|--#########--|--#########--
|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--
|--##-----##-

In [21]:
totaltext = concatenate_scrambled_diluted(xi_s5, 10, W, 0.1)
print(totaltext)

|-###-#--#----|#-#--#######-|##--#-0-##---|#-######-#--#|#######-#####|#---###-#-###|#--#--#######|-###--#--##-#|------#--#--#|#-#####-###-#
|##-##---##-#-|#--###-###-##|###--#-#-#--#|#-##-##-#--#-|####-----##--|####0#-#-####|###-#-###-#--|#--##-0-####0|#-#-0--###-#0|#####--#--###
|######--####-|-###--#--0###|-####-##----#|###--###-#--#|####--#-#-###|--#-#-#---##-|##-#-#--#-#--|#-##--#----##|-#######-####|####--####--#
|##---####-##-|-##--#----##-|-##--###-####|-####-#---##-|#-##-##---#--|###--#---#0-#|###-#-###-###|#--#----#----|###---##--#--|--#-#--##---#
|##----###--##|#---#-0----#-|###--#######-|#-##---##-###|-#-#######-#-|-##-#-##---##|##------#-#-#|#---##-##--##|#0-#-###-#-##|#-#-#0#-----#
|##0#-#----##-|###-#--#---#-|--#----##-#-#|##---#-###---|#----#-#--##-|--##-#-#---#-|--#-#--###-##|-#-------#-#-|###-##-##-#-#|-#-----#---##
|###-0-#--#-##|#-###--####--|###-##-#-----|-----####--##|-##-#-#-##-#-|0#--#-##---##|-#-#-##---#-#|######--#-###|#-#-####---#-|##--#-###--#-
|----#-#-#-#-

In [22]:
totaltext = concatenate_scrambled_diluted(xi_s9, 10, W, 0.1)
print(totaltext)

|#############|#############|#############|#############|#############|#############|#############|#############|#############|#############
|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####
|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####
|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###
|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##
|##---------##|##---------##|##---------##|##---------##|##---------##|##---------##|##---------##|##---------##|##---------##|##---------##
|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##
|##--#####--#

#### Medium dilution, scrambled arrays:

In [23]:
totaltext = concatenate_scrambled_diluted(xi_s1, 10, W, 0.5)
print(totaltext)

|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------
|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----
|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----
|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---
|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--
|--#########--|--#########--|--#########--|--#########--|--#########--|--#########--|--#########--|--#########--|--#########--|--#########--
|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--
|--##-----##-

In [24]:
totaltext = concatenate_scrambled_diluted(xi_s5, 10, W, 0.5)
print(totaltext)

|-#####-#0###-|####-##--#-#-|#####--##-#-0|#-###--##---#|#-#0#-##--#0-|#--#-###--###|#-#-##-----##|-#------#--#0|#--#-###-#-#0|#####---###-#
|-######-#0#-#|-#######--#-#|--###--0----#|-#-####-##-#-|#--##-#-#-###|--##--#-##-##|--###-#-#-###|#---###-#####|#-###---##---|###--#-#-####
|-###--#---###|--###-##-#-##|--#-#--#-#-##|-###-#-0####0|##---0----#--|#-#-###-##-##|######-#----#|-######0--#0-|##-#-##-#--##|-#--#-####--#
|#-#--#-#-##--|--###0-##-###|0##0-#-#-#--#|-#0#--###--#-|##---#-##--##|---#-#---####|---####----#-|--#-####-#-##|--#-0##-###-#|--#---#-#0-#0
|###---#-#---#|#----#----#--|###---##-#---|##-##-#-#-#--|#-#0#-#-##---|#--#-###-##-#|#----#-#0-###|####-####----|#-#---#-#-###|--#---#-##-#-
|-####0-#---##|#---#-#-##--#|#-#-##--0--0#|--##0###---##|##-#--###--##|-0-##---##--#|--#--#-0##0#-|#-#--#------#|###-#------#-|###--##---###
|--#-#--#---#-|#-###---#-#-#|###-#-##--###|-##---#####--|----#-0##-0--|--#-###--#-#-|#--###----###|---########0#|##--##--#---#|--#-#-###--0-
|-#---###-#--

In [25]:
totaltext = concatenate_scrambled_diluted(xi_s9, 10, W, 0.5)
print(totaltext)

|#############|#############|#############|#############|#############|#############|#############|#############|#############|#############
|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####
|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####
|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###
|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##
|##---------##|##---------##|##---------##|##---------##|##---------##|##---------##|##---------##|##---------##|##---------##|##---------##
|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##
|##--#####--#

#### Strong dilution, scrambled arrays:

In [26]:
totaltext = concatenate_scrambled_diluted(xi_s1, 10, W, 0.9)
print(totaltext)

|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------
|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----|-----###-----
|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----|----##-##----
|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---|---##---##---
|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--
|--#########--|--#########--|--#########--|--#########--|--#######0#--|--#########--|--#########--|--#########--|--#########--|--#########--
|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--|--##-----##--
|--##-----##-

In [27]:
totaltext = concatenate_scrambled_diluted(xi_s5, 10, W, 0.9)
print(totaltext)

|-----#-#--#0-|#0#-0###-0#--|#-0###-#-#-0#|----#---0-#-#|##-###-#-#-#-|#--#00#0--##-|#-#--0--#----|####--#-0#-#-|--#--0#-#####|-0-#-#####--0
|0--#--###-0--|#-#00###-#00#|##---0#-#-###|#-0##-#-#-0##|-----###--##-|--00-#-0--00#|#####---##-##|-##0#-#-#-#--|-0#-####-#-##|#-#0####--###
|---0---#---0#|0-#000#0-####|-##--0-0-####|000#---#0-##-|#00-##0-0-#-#|###-0###-###-|-----#--#-0##|0-##0-###-##-|###-##------0|#-##-###--#--
|0-#---#--#-#0|##0-##-##-#--|----#-00#--0-|0--#0--#--#--|#-##-####0--#|#-#-0#00#-#-0|-0#-###0-----|-0##--#--####|-#---#--##-#-|####0---0#--#
|###-#-#-----#|0--0#-----0-#|#--#--#--##-#|###########--|--##-###---#-|0#-#-00#--##0|#-##-####-###|#-#0-#---#-##|#-#--####-##0|#-#-#-#-#-#-#
|#----####-###|0--###------0|-#----#-#0-#-|#---#--##-##-|##-##0#0-##0-|-#--#-#--##0#|-#-#-0#---#--|-##0##-0---#0|#--#-###--#-#|#-0--####---#
|#-###0#-0##-0|#--0--##-#-##|#----#-0----0|----0###---0#|-0-#--##--0#0|---##-#-#-#-0|##0#0--#--0-0|-#--#####-##-|0#-###--0#-##|-#-#-0-#####-
|##-0---#####

In [28]:
totaltext = concatenate_scrambled_diluted(xi_s9, 10, W, 0.9)
print(totaltext)

|#############|#############|#############|#############|#############|#############|#############|#############|#############|#############
|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####|#####---#####
|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####|####--#--####
|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###|###--###--###
|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##
|##---------##|##---------##|##---------##|##---------##|##---------##|##---------##|##---------##|##---------##|##-------0-##|##---------##
|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##|##--#####--##
|##--#####--#

It should be noted that in the 0.5 cases, the addition of zeros in the weights results in inconclusive sign(x) functions, which are not translated to a - or # character, but to a 0 character.

In [29]:
def concatenate_random_diluted(xi, N, W, c, H=9):
    '''Concatenate the output of several diluted networks with random inputs'''
    
    pat = [''] * H
    
    for i in range(N):
        xi_r = random_input(xi)
        xi_new = Hopfield_diluted(xi_r, W, c)
        
        text = decode_pattern(xi_new)
        
        text_split = text.split('\n')
        
        for line in range(H):
            pat[line] += '|'
            pat[line] += text_split[line]
        
        
        
        totaltext = '\n'.join(pat)
        
    return totaltext
    

#### Weak dilution, random arrays:

In [30]:
pat = concatenate_random_diluted(xi, 10, W, 0.1)
print(pat)

|-------------|------#------|#############|#############|###-----0#-#-|#############|#############|#--##########|-####-#-###0-|-------------
|-----#-#----#|##--####-#---|#####---#####|#####---#####|#-#-#-0#-----|#####---#####|#####---#####|##-#-##--####|#######-#####|-----###-----
|----###0#----|----#--##----|##-#--#--####|####--#--####|-##-##--#----|####--#--####|####--#--####|##########-#-|####--#---###|----##-##----
|---##---##---|---##----#---|###--###--###|###--###--###|-#-###--##---|###--###--###|###--###--###|0-##--##---##|-#-#-###--###|---##---##---
|--0#-----##--|--#------##--|##--#####--##|##--#####--##|-######0--#--|##--#####--##|##--##-##--##|#0--####---##|#--####-#--##|--##-----##--
|--###-#####--|--#########--|##---------##|##---------##|#-###--#-##--|##---------##|##---------##|-###-##--####|###-#-#---###|--#########--
|-###-----##--|--###-0--##--|##--#####--##|##---####---#|-###-##-#-0--|##--#####--##|##--#####--##|#######-#-###|##--######-#-|--##-----##--
|--###----##-

#### Medium dilution, random array:

In [31]:
pat = concatenate_random_diluted(xi, 10, W, 0.5)
print(pat)

|0--###-#---##|-------------|#-#--###-###-|--#---------#|-------------|-------------|---0--0##-#--|-------------|##0###00-#0#-|0##--#-######
|##0###-###---|-----###-----|###-#--####--|--#---##---#-|-#---###-----|-----###-----|--###-#######|-----###-----|######-######|###-#--###-##
|#-###-#0-####|----##-##----|#-#-#--#####-|#--###--#--#-|----##-##----|-#-###-##----|-#-#-----#-#-|----##-##----|##-#---######|#####----#---
|##---##0#0-#0|---##---##---|###--####-#-#|#--##-0-##0-#|---##--###---|---##---##---|##--#---0-0#-|---##---#----|#-#---#-#-###|--#0####--##-
|-#-##----#---|--##-----##--|##-#-#####-##|####-#---#--#|--##-----##--|--##-----##0-|-##-0#---##-#|--##-----##--|----##----###|-#-###-##-##-
|------##-##-#|--#########--|##--0-##-----|--#-#-###-#--|-#-0#0#####--|--##-######--|#-----##--##-|--#########--|#-#------#-#-|-###--#--#---
|##0--##-#-#-#|--##-----##--|##---####--##|#--#-#-#-0#-#|--##-----##--|--##--#--#---|#0#-#--###--#|--##-----##--|###-#####--##|###-###0-#--#
|###-#######-

#### Strong dilution, random array:

In [32]:
pat = concatenate_random_diluted(xi, 10, W, 0.9)
print(pat)

|###-###0####-|-##-######-##|0-#----###---|--##----000##|#-0#----0----|0-######-#-##|-#-0#-0-##-#-|-0-#####-#-#-|-###-##-#-##-|#0####---##0-
|--#---#--##-0|0#--#0#---###|-###-0##---##|-#-#0#0-##---|----###-#-00-|###0#-#-#-0##|-0-#-0#0#00#-|---##--0##0-#|##0##--#-----|#---0##0-#-#-
|0-###-#0##-#0|#-##--###0###|#----####0##-|-##-####--#-0|#--#--#0#-#-0|00-----##--#0|#00#-0#---00-|--#--------##|---0----0###-|##--##0#00-0-
|##---##---0##|-0##0-##---#0|-0-#0#-----#-|-0-##---##-##|-0-#--00#0---|###-0##-----#|0#--##-####-#|#00--0-------|-#--0#0-##-##|-#0-##-#-#--#
|0###-0-##0--#|0--###-##--#0|##--#0---#-0-|-0-#-00-0##--|-0-#--##-##--|0#0-###--#---|--#-##-#-0--#|#---0###--0--|####-#0-#-#00|#-0####--#-##
|0##-----0---#|#-----0---###|#----#-######|--##-##--##-#|-###-##-#-#0#|##-##-#-#---#|#0-#-#-##-###|-#---0#0----#|##----##---#0|0--##-0--#--#
|#0-----0---##|#--#0#-#--###|-#####--#####|-00-###-####-|-####---###--|###-#--##0-##|#-0-#-#0###--|#0###0----0##|####---0-#---|#-----#0##--#
|####-#-#0-#-